In [1]:
import pandas as pd
import sqlite3
import os

# Path to CSV folder and output database
csv_dir = "/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/csv"
db_path = os.path.join(csv_dir, "healthcare.db")

# Load CSV files
patients_df = pd.read_csv(os.path.join(csv_dir, "patients.csv"))
medications_df = pd.read_csv(os.path.join(csv_dir, "medications.csv"))
prescriptions_df = pd.read_csv(os.path.join(csv_dir, "prescriptions.csv"))
visits_df = pd.read_csv(os.path.join(csv_dir, "visits.csv"))

# Connect to SQLite DB (will create if it doesn't exist)
conn = sqlite3.connect(db_path)

# Write data to SQLite
patients_df.to_sql("patients", conn, if_exists="replace", index=False)
medications_df.to_sql("medications", conn, if_exists="replace", index=False)
prescriptions_df.to_sql("prescriptions", conn, if_exists="replace", index=False)
visits_df.to_sql("visits", conn, if_exists="replace", index=False)

# Confirm tables are created
print("Database created and tables loaded successfully!")

# Optional: close connection
conn.close()


Database created and tables loaded successfully!


In [9]:
import os
from langchain.agents.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv

# Load OpenAI key from .env file
load_dotenv()  # Make sure .env contains OPENAI_API_KEY

# Path to your SQLite database
db_path = "/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/healthcare.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")  # Schema auto-loaded

# Initialize GPT-3.5 model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
)

# Create SQL Agent that knows how to talk to the database
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    prefix="""
You are a helpful medical data assistant.

Database schema:
- `patients` stores patient_id, name, age, gender
- `visits` stores visit_id, patient_id, date, reason
- `prescriptions` links visits and medications with dosage info
- `medications` stores med_id, name, and category

🧠 When a user asks about medical conditions like 'hypertension' or 'chest pain',
they are stored in the `visits.reason` column.

Use this to filter visits, and join with `patients` on `patient_id`.
The goal is to accurately represent the relationships between patients, visits, medications, and prescriptions.

📝 When summarizing a patient's history, include:
- Patient name, age, and gender
- Visits and medical reasons
- Medications prescribed with dosage and category

Format your response as a **natural paragraph**, like a short clinical note. Do NOT use bullet points.
"""
)




# Run chatbot loop
print("📋 SQL ChatBot is ready. Ask questions or type 'exit'.")
while True:
    query = input("\n🧠 Your question: ")
    if query.lower() in ("exit", "quit"):
        print("👋 Goodbye!")
        break
    try:
        response = agent_executor.run(query)
        print(f"\n✅ Answer:\n{response}")
    except Exception as e:
        print(f"\n❌ Error: {e}")


📋 SQL ChatBot is ready. Ask questions or type 'exit'.


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


medications, patients, prescriptions, visits
Invoking: `sql_db_schema` with `{'table_names': 'patients, visits, prescriptions, medications'}`



CREATE TABLE medications (
	med_id INTEGER, 
	name TEXT, 
	category TEXT
)

/*
3 rows from medications table:
med_id	name	category
1	Atenolol	Cardiac
2	Lisinopril	Cardiac
3	Metformin	Diabetes
*/


CREATE TABLE patients (
	patient_id INTEGER, 
	name TEXT, 
	age INTEGER, 
	gender TEXT
)

/*
3 rows from patients table:
patient_id	name	age	gender
1	Michael Pierce	62	Female
2	David Williams	78	Female
3	Michael Chambers	42	Other
*/


CREATE TABLE prescriptions (
	id INTEGER, 
	visit_id INTEGER, 
	med_id INTEGER, 
	dosage TEXT
)

/*
3 rows from prescriptions table:
id	visit_id	med_id	dosage
1	1610	6	50mg
2	835	7	50mg
3	1000	2	50mg
*/


CREATE TABLE visits (
	visit_id INTEGER, 
	patient_id INTEGER, 
	date TEXT,